In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

data = pd.read_csv("final_output.csv")

In [ ]:
data.head(5)

In [ ]:
# Import the module
from sklearn.preprocessing import LabelEncoder

# Create an instance of the class
le = LabelEncoder()

# Fit the encoder on the column
le.fit(['A', 'H', 'D'])

# Check the classes and their corresponding labels
print(le.classes_)
# Output: array(['A', 'D', 'H'], dtype='<U1')

# Transform the column into numerical labels
data['FTR'] = le.transform(data['FTR'])
print(data['FTR'])
# Output: array([2, 0, 1, 1, 2, ...])



In [ ]:
#Splitting the data into independent and dependent variables
X = data.iloc[:,0:7].values
y = data.iloc[:,7].values
print('The independent features set: ')
print(X[:7,:])
print('The dependent variable: ')
print(y[:7])

In [ ]:
X = X.astype('float32')
X.dtype

In [ ]:
X.shape

In [ ]:
import torch


y = torch.tensor(y,dtype=torch.long)

In [ ]:
y.shape

In [ ]:
# Split the data to train and test dataset.
from sklearn.model_selection import train_test_split
sample_weight = np.random.RandomState(42).rand(y.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=True,random_state=42)

In [ ]:
X_train.shape

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence


The Transformer Model

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,num_heads):
        super(TransformerModel,self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads), num_layers)
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.fc(x)

        return x



In [ ]:
#set random seeds
torch.manual_seed(42)

#creating a data loader
# set up DataLoader for training set
train_loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)

# set up DataLoader for testing set
test_loader = DataLoader(list(zip(X_test, y_test)), shuffle=True, batch_size=1)

#define hyperparameters
input_dim = 7
hidden_dim = 64
output_dim = 3
num_layers = 3
num_heads = 4
batch_size = 32
num_epochs = 50
learning_rate = 0.001

#create models and optimizers
model = TransformerModel(input_dim,hidden_dim, output_dim, num_layers, num_heads)

optimizer =  optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()


In [ ]:
#define hyperparameters
for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        #adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item() 
        _,predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * total_correct / total_samples

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


In [ ]:
#evaluating the model on a test set
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        #adjusting the dimensionality of input data
        inputs = inputs.unsqueeze(1)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100* correct/ total
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
#["FTHG","FTAG","HTHG","HTAG","Form","B365H","B365D","B365A"]
X_new = np.array([[1.08,1.08,1.00,2.90,3.60,2.30]])
#X_net = torch.FloatTensor(X_new)
#X_new.view(-1,1)
X_new = torch.IntTensor(X_new)
X_new.shape

In [ ]:
import torch

# Assuming your model, optimizer, and criterion are already defined
# model = TransformerModel(input_dim, hidden_dim, output_dim, num_layers, num_heads)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# Load the trained model checkpoint
#checkpoint = torch.load('path_to_your_model_checkpoint.pth')
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']

# Set the model to evaluation mode
model.eval()

# Create a new input tensor with 7 elements
new_input = torch.tensor([2.79,2.58,3.53,3.32,2.25,3.30,3.20])

# Adjust the dimensionality of the input data
new_input = new_input.unsqueeze(0).unsqueeze(1)  # Adding batch and sequence dimensions

# Make the prediction
with torch.no_grad():
    prediction = model(new_input)

# Convert the prediction to a probability distribution using softmax
predicted_probs = torch.softmax(prediction, dim=1)

# Get the predicted class
predicted_class = torch.argmax(predicted_probs, dim=1).item()

print("Predicted Class:", predicted_class)
#{0:A, 1:D, 2:H}


In [ ]:
import pickle

#Create Pickle file from the Neural network Model
with open('transformer.pickle', 'wb') as dump_var:
   pickle.dump(data, dump_var)

In [ ]:

#[0.95,1.30,2.29,3.10,2.20,3.40,3.30]
#[0.90,1.33,2.29,3.24,2.45,3.30,2.90]
#[1.17,1.06,3.28,2.17,2.90,3.10,2.62]
#[0.91,0.96,2.30,1.91,2.45,3.00,3.10]
#[1.61,1.72,3.33,2.78,3.40,3.40,2.15]
#[1.33,1.00,2.61,2.22,2.30,3.30,3.20]
#[1.20,1.50,2.20,2.75,2.25,3.10,3.50]
#[1.24,1.25,2.90,2.80,2.05,3.40,3.60]
#[1.90,2.19,2.95,3.19,3.10,3.50,2.15]
#[1.30,1.17,2.57,2.70,2.50,3.10,3.00]
#[2.79,2.58,3.53,3.32,2.25,3.30,3.20]